Table 4.12.2: Number of children living with individuals starting treatment 2017-18

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['Table 4.12.2']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
cell = tab.filter('Number of children')
observations = tab.filter('Total').fill(DOWN).is_not_blank().is_not_whitespace().is_number()
children = cell.expand(DOWN).is_not_blank().is_not_whitespace()
Clients = cell.expand(RIGHT).is_not_blank().is_not_whitespace().is_not_number()
Dimensions = [
            HDim(children,'Treatment group',DIRECTLY,LEFT),
            HDim(Clients,'Clients in treatment',CLOSEST,LEFT),
            HDimConst('Measure Type','Count'),
            HDimConst('Unit','Clients in treatment'),
            HDimConst('Period', '2018-03-31T00:00:00/P1Y'),
            HDimConst('Basis of treatment','number of children')
            ]

In [4]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [5]:
new_table = c1.topandas()

In [6]:
import numpy as np
new_table['OBS'].replace('', np.nan, inplace=True)
new_table.dropna(subset=['OBS'], inplace=True)
new_table.rename(columns={'OBS': 'Value'}, inplace=True)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Value'] = new_table['Value'].map(lambda x:''
                                  if (x == '-')
                                  else int(x))

In [7]:
new_table['Treatment group'] = new_table['Treatment group'].map(
    lambda x: {
        'Number of children' : 'Number of children living with people in treatment',
        'Number of individuals living with children' : 'Number of clients living with children'
        }.get(x, x))

In [8]:
new_table = new_table[['Period','Basis of treatment','Treatment group','Clients in treatment','Measure Type','Value','Unit']]

In [9]:
new_table.head()

,Period,Basis of treatment,Treatment group,Clients in treatment,Measure Type,Value,Unit
0,2018-03-31T00:00:00/P1Y,number of children,Number of children living with people in treat...,Opiate,Count,10328,Clients in treatment
1,2018-03-31T00:00:00/P1Y,number of children,Number of children living with people in treat...,Non-opiate only,Count,7424,Clients in treatment
2,2018-03-31T00:00:00/P1Y,number of children,Number of children living with people in treat...,Non-opiate and Alcohol,Count,7453,Clients in treatment
3,2018-03-31T00:00:00/P1Y,number of children,Number of children living with people in treat...,Alcohol only,Count,20904,Clients in treatment
4,2018-03-31T00:00:00/P1Y,number of children,Number of children living with people in treat...,Total,Count,46109,Clients in treatment
